In [36]:
from googletrans import Translator
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd

In [18]:
translator = Translator()
scielo = load_dataset("scielo", "en-es")

Reusing dataset scielo (/Users/anon/.cache/huggingface/datasets/scielo/en-es/1.0.0/9e049789539b98dda0fbc7503d937a3c2c18758f6cab6f8d2b13b46cd18f0fa8)


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
scielo

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 177782
    })
})

In [45]:
SAMPLE_SIZE = 1000
scielo_shuffled = scielo.shuffle(seed=42)
scielo_sampled = scielo_shuffled['train'].select(range(SAMPLE_SIZE))

Loading cached shuffled indices for dataset at /Users/anon/.cache/huggingface/datasets/scielo/en-es/1.0.0/9e049789539b98dda0fbc7503d937a3c2c18758f6cab6f8d2b13b46cd18f0fa8/cache-0057d846ff1541c2.arrow


In [46]:
es = [translation['es'] for translation in scielo_sampled['translation']]
en = [translation['en'] for translation in scielo_sampled['translation']]

In [47]:
translations = []
for sample in tqdm(es):
    translations_raw = translator.translate(sample, dest='en')
    translations.append(translations_raw.text)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [53]:
from datasets import load_metric
labels = [[label or "noop"] for label in en]
metric = load_metric("sacrebleu")
result = metric.compute(predictions=translations, references=labels)

In [54]:
print(result['score'])

40.90407300241927


In [59]:
df = pd.DataFrame({
    "text": translations + en,
    "fake": [1 for translation in translations] + [0 for text in en]
})
df.to_csv('../data/fake_papers_translated.csv')

In [60]:
df['text'].apply(lambda s: len(s.split())).describe()

count    2000.000000
mean       29.785000
std        18.556264
min         0.000000
25%        18.000000
50%        27.000000
75%        37.000000
max       226.000000
Name: text, dtype: float64